In [1]:
import os
import time
import glob
import re
import pandas as pd
from functools import reduce
from klepto.archives import dir_archive

import numpy as np
from tqdm.auto import tqdm
import coffea.processor as processor
from coffea.processor.accumulator import AccumulatorABC
from coffea.analysis_objects import JaggedCandidateArray
from coffea.btag_tools import BTagScaleFactor
from coffea import hist
import pandas as pd
import uproot_methods
import uproot
import awkward
import copy

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from Tools.config_helpers import *
from Tools.helpers import mergeArray, mt, get_scheduler_address

from Tools.objects import Collections
from Tools.cutflow import Cutflow

# This just tells matplotlib not to open any
# interactive windows.
matplotlib.use('Agg')

In [2]:
def pad_and_flatten(val): 
    try:
        return val.pad(1, clip=True).fillna(0.).flatten()#.reshape(-1, 1)
    except AttributeError:
        return val.flatten()

#os.environ['KERAS_BACKEND'] = 'theano'
#from keras.models import load_model

In [3]:
import sys
sys.setrecursionlimit(10000)
print(sys.getrecursionlimit())

10000


In [4]:
#global variables... to avoid making bugs!!!
processesList = ['DYJets', 'TTJets', 'ttW', 'ttZ', 'Data']
linesList= ['triggers', 'dilepton sf']

In [11]:
class analysisProcessor(processor.ProcessorABC):
    """Processor used for running the analysis"""
    def __init__(self):
        
        ## load b-tag SFs
        #self.btag_sf = BTagScaleFactor(os.path.expandvars("$TWHOME/data/DeepCSV_102XSF_V1.btag.csv.gz", "reshape")

        ## load the NN
        #self.model = load_model('../ML/data/training.h5')
        #self.stds  = pd.read_json('../ML/data/stds.json').squeeze()
        #self.means = pd.read_json('../ML/data/means.json').squeeze()
        
        # we can use a large number of bins and rebin later
        dataset_axis        = hist.Cat("dataset",   "Primary dataset")
        pt_axis             = hist.Bin("pt",        r"$p_{T}$ (GeV)", 1000, 0, 1000)
        p_axis              = hist.Bin("p",         r"$p$ (GeV)", 1000, 0, 2500)
        ht_axis             = hist.Bin("ht",        r"$H_{T}$ (GeV)", 500, 0, 5000)
        mass_axis           = hist.Bin("mass",      r"Mass (GeV)", 1000, 0, 2000)
        eta_axis            = hist.Bin("eta",       r"$\eta$", 60, -5.5, 5.5)
        delta_axis          = hist.Bin("delta",     r"$\delta$", 100,0,10 )
        multiplicity_axis   = hist.Bin("multiplicity",         r"N", 20, -0.5, 19.5)
        norm_axis           = hist.Bin("norm",         r"N", 25, 0, 1)

        self._accumulator = processor.dict_accumulator({
            "Z_mass":                  hist.Hist("Counts", dataset_axis, mass_axis),
            #"MET_pt_baseline" :          hist.Hist("Counts", dataset_axis, pt_axis),
            #"HT_baseline" :              hist.Hist("Counts", dataset_axis, ht_axis),
            #"mtb_min_baseline" :         hist.Hist("Counts", dataset_axis, mass_axis),
            #"MET_pt" :          hist.Hist("Counts", dataset_axis, pt_axis),
            #"HT" :              hist.Hist("Counts", dataset_axis, ht_axis),
            #"mtb_min" :         hist.Hist("Counts", dataset_axis, mass_axis),
            #"MET_pt_CR" :       hist.Hist("Counts", dataset_axis, pt_axis),
            #"HT_CR" :           hist.Hist("Counts", dataset_axis, ht_axis),
            #"mtb_min_CR" :      hist.Hist("Counts", dataset_axis, mass_axis),
            #"lead_AK8_pt" :     hist.Hist("Counts", dataset_axis, pt_axis),
            #"W_pt" :            hist.Hist("Counts", dataset_axis, pt_axis),
            #"H_pt" :            hist.Hist("Counts", dataset_axis, pt_axis),
            #"W_eta" :           hist.Hist("Counts", dataset_axis, eta_axis),
            #"H_eta" :           hist.Hist("Counts", dataset_axis, eta_axis),
            
#             "met_CR":           hist.Hist("Counts", dataset_axis, pt_axis),
#             "met_Higgs_CR":     hist.Hist("Counts", dataset_axis, pt_axis),
#             "met_W_CR":         hist.Hist("Counts", dataset_axis, pt_axis),
#             "met_Higgs_W_CR":   hist.Hist("Counts", dataset_axis, pt_axis),
            
#             "ht_CR":            hist.Hist("Counts", dataset_axis, ht_axis),
#             "ht_Higgs_CR":      hist.Hist("Counts", dataset_axis, ht_axis),
#             "ht_W_CR":          hist.Hist("Counts", dataset_axis, ht_axis),
#             "ht_Higgs_W_CR":    hist.Hist("Counts", dataset_axis, ht_axis),
            
#             "N_AK8_CR" :        hist.Hist("Counts", dataset_axis, multiplicity_axis),
#             "W_pt_CR" :         hist.Hist("Counts", dataset_axis, pt_axis),
#             "H_pt_CR" :         hist.Hist("Counts", dataset_axis, pt_axis),
#             "W_eta_CR" :        hist.Hist("Counts", dataset_axis, eta_axis),
#             "H_eta_CR" :        hist.Hist("Counts", dataset_axis, eta_axis),
            
#             "N_AK8_Higgs_CR" :  hist.Hist("Counts", dataset_axis, multiplicity_axis),
#             "W_pt_Higgs_CR" :   hist.Hist("Counts", dataset_axis, pt_axis),
#             "H_pt_Higgs_CR" :   hist.Hist("Counts", dataset_axis, pt_axis),
#             "W_eta_Higgs_CR" :  hist.Hist("Counts", dataset_axis, eta_axis),
#             "H_eta_Higgs_CR" :  hist.Hist("Counts", dataset_axis, eta_axis),
            
#             "N_AK8_W_CR" :      hist.Hist("Counts", dataset_axis, multiplicity_axis),
#             "W_pt_W_CR" :       hist.Hist("Counts", dataset_axis, pt_axis),
#             "H_pt_W_CR" :       hist.Hist("Counts", dataset_axis, pt_axis),
#             "W_eta_W_CR" :      hist.Hist("Counts", dataset_axis, eta_axis),
#             "H_eta_W_CR" :      hist.Hist("Counts", dataset_axis, eta_axis),
            
#             "N_AK8_Higgs_W_CR" :hist.Hist("Counts", dataset_axis, multiplicity_axis),
#             "W_pt_Higgs_W_CR" : hist.Hist("Counts", dataset_axis, pt_axis),
#             "H_pt_Higgs_W_CR" : hist.Hist("Counts", dataset_axis, pt_axis),
#             "W_eta_Higgs_W_CR" :hist.Hist("Counts", dataset_axis, eta_axis),
#             "H_eta_Higgs_W_CR" :hist.Hist("Counts", dataset_axis, eta_axis),

            #"N_b" :             hist.Hist("Counts", dataset_axis, multiplicity_axis),
            #"N_AK4" :           hist.Hist("Counts", dataset_axis, multiplicity_axis),
            #"N_AK8" :           hist.Hist("Counts", dataset_axis, multiplicity_axis),
            #"N_H" :             hist.Hist("Counts", dataset_axis, multiplicity_axis),
            #"N_W" :             hist.Hist("Counts", dataset_axis, multiplicity_axis),
            
            #"WH_deltaPhi":      hist.Hist("Counts", dataset_axis, delta_axis),
            #"WH_deltaR":        hist.Hist("Counts", dataset_axis, delta_axis),
            #"bb_deltaPhi":      hist.Hist("Counts", dataset_axis, delta_axis),
            #"bb_deltaR":        hist.Hist("Counts", dataset_axis, delta_axis),
            #"min_dphiJetMet4":  hist.Hist("Counts", dataset_axis, delta_axis),
            #"dphiDiJet":        hist.Hist("Counts", dataset_axis, delta_axis),
            #"dphiDiFatJet":     hist.Hist("Counts", dataset_axis, delta_axis),
            
#             'mC750_l1':         processor.defaultdict_accumulator(int),
#             'WJets':            processor.defaultdict_accumulator(int),
#             'QCD':              processor.defaultdict_accumulator(int),
            'TTJets':           processor.defaultdict_accumulator(int),
            'DYJets':           processor.defaultdict_accumulator(int),
#             'ZNuNu':            processor.defaultdict_accumulator(int),
#             'ST':               processor.defaultdict_accumulator(int),
#             'ST_tW':            processor.defaultdict_accumulator(int),
#             'ST_tChannel':      processor.defaultdict_accumulator(int),
#             'ST_sChannel':      processor.defaultdict_accumulator(int),
            'ttW':              processor.defaultdict_accumulator(int),
            'ttZ':              processor.defaultdict_accumulator(int),
#             'WW':               processor.defaultdict_accumulator(int),
#             'WZ/ZZ':            processor.defaultdict_accumulator(int),
#             'LL':               processor.defaultdict_accumulator(int),
            'Data':             processor.defaultdict_accumulator(int),
            'totalEvents':      processor.defaultdict_accumulator(int),
#             'test1':            processor.defaultdict_accumulator(float),
        })

    @property
    def accumulator(self):
        return self._accumulator

    def process(self, df):
        """
        Processing function. This is where the actual analysis happens.
        """
        output = self.accumulator.identity()
        dataset = df["dataset"]
        cfg = loadConfig()
        
        ############## MET ##############
        
        met_pt  = df["MET_pt"]
        met_phi = df["MET_phi"]
        
        ############## LEPTONS ############## 
        
        muon = JaggedCandidateArray.candidatesfromcounts(
            df['nMuon'],
            pt = df['Muon_pt'].content,
            eta = df['Muon_eta'].content,
            phi = df['Muon_phi'].content,
            mass = df['Muon_mass'].content,
            miniPFRelIso_all=df['Muon_miniPFRelIso_all'].content,
            looseId =df['Muon_looseId'].content
            )
        muon = muon[(muon.pt > 10) & (abs(muon.eta) < 2.4) & (muon.looseId) & (muon.miniPFRelIso_all < 0.2)]
        #muon = Collections(df, "Muon", "tightTTH").get() # this needs a fix for DASK
        
        electron = JaggedCandidateArray.candidatesfromcounts(
            df['nElectron'],
            pt = df['Electron_pt'].content,
            eta = df['Electron_eta'].content,
            phi = df['Electron_phi'].content,
            mass = df['Electron_mass'].content,
            miniPFRelIso_all=df['Electron_miniPFRelIso_all'].content,
            cutBased=df['Electron_cutBased'].content
            )
        electron = electron[(electron.pt>10) & (abs(electron.eta) < 2.4) & (electron.miniPFRelIso_all < 0.1) &  (electron.cutBased >= 1)]
        #electron = Collections(df, "Electron", "tightTTH").get() # this needs a fix for DASK

        high_pt_e = electron[electron.pt.argsort(ascending=False)][:,:2]
        ee = high_pt_e.choose(2)
        
        high_pt_m = muon[muon.pt.argsort(ascending=False)][:,:2]
        mm = high_pt_m.choose(2)
        mm_mass = mm.mass


        
        ############## FATJETS ##############
        
        fatjet = JaggedCandidateArray.candidatesfromcounts(
            df['nFatJet'],
            pt = df['FatJet_pt'].content,
            eta = df['FatJet_eta'].content,
            phi = df['FatJet_phi'].content,
            mass = df['FatJet_mass'].content,
            msoftdrop = df["FatJet_msoftdrop"].content,  
            deepTagMD_HbbvsQCD = df['FatJet_deepTagMD_HbbvsQCD'].content, 
            deepTagMD_WvsQCD = df['FatJet_deepTagMD_WvsQCD'].content, 
            deepTag_WvsQCD = df['FatJet_deepTag_WvsQCD'].content
            
        )
        
        leadingFatJets = fatjet[:,:2]
        difatjet = leadingFatJets.choose(2)
        dphiDiFatJet = np.arccos(np.cos(difatjet.i0.phi-difatjet.i1.phi))
        
        htag = fatjet[((fatjet.pt > 200) & (fatjet.deepTagMD_HbbvsQCD > 0.8365))]
        htag_hard = fatjet[((fatjet.pt > 300) & (fatjet.deepTagMD_HbbvsQCD > 0.8365))]
        
        lead_htag = htag[htag.pt.argmax()]
        
        wtag = fatjet[((fatjet.pt > 200) & (fatjet.deepTagMD_HbbvsQCD < 0.8365) & (fatjet.deepTag_WvsQCD > 0.918))]
        wtag_hard = fatjet[((fatjet.pt > 300) & (fatjet.deepTagMD_HbbvsQCD < 0.8365) & (fatjet.deepTag_WvsQCD > 0.918))]
        
        lead_wtag = wtag[wtag.pt.argmax()]
        
        wh = lead_htag.cross(lead_wtag)
        wh_deltaPhi = np.arccos(wh.i0.phi - wh.i1.phi)
        wh_deltaR = wh.i0.p4.delta_r(wh.i1.p4)
        
        ############## JETS ##############
        
        jet = JaggedCandidateArray.candidatesfromcounts(
            df['nJet'],
            pt = df['Jet_pt'].content,
            eta = df['Jet_eta'].content,
            phi = df['Jet_phi'].content,
            mass = df['Jet_mass'].content,
            jetId = df['Jet_jetId'].content, # https://twiki.cern.ch/twiki/bin/view/CMS/JetID
            #puId = df['Jet_puId'].content, # https://twiki.cern.ch/twiki/bin/viewauth/CMS/PileupJetID
            btagDeepB = df['Jet_btagDeepB'].content, # https://twiki.cern.ch/twiki/bin/viewauth/CMS/BtagRecommendation102X
            #deepJet = df['Jet_'].content # not there yet?
        )
        
        skimjet   = jet[(jet.pt>30) & (abs(jet.eta)<2.4)]
        jet       = jet[(jet.pt>30) & (jet.jetId>1) & (abs(jet.eta)<2.4)]
        jet       = jet[~jet.match(muon, deltaRCut=0.4)] # remove jets that overlap with muons
        jet       = jet[~jet.match(electron, deltaRCut=0.4)] # remove jets that overlap with electrons
        jet       = jet[~jet.match(fatjet, deltaRCut=1.2)] # remove AK4 jets that overlap with AK8 jets
        jet       = jet[jet.pt.argsort(ascending=False)] # sort the jets
        btag      = jet[(jet.btagDeepB>0.4184)]
        light     = jet[(jet.btagDeepB<0.4184)]
        
        ## Get the leading b-jets
        high_score_btag = jet[jet.btagDeepB.argsort(ascending=False)][:,:2]
        
        leading_jet    = jet[jet.pt.argmax()]
        leading_b      = btag[btag.pt.argmax()]
        
        bb = high_score_btag.choose(2)
        bb_deltaPhi = np.arccos(np.cos(bb.i0.phi-bb.i1.phi))
        bb_deltaR = bb.i0.p4.delta_r(bb.i1.p4)
        
        mtb = mt(btag.pt, btag.phi, met_pt, met_phi)
        min_mtb = mtb.min()
        mth = mt(htag.pt, htag.phi, met_pt, met_phi)

        ############## OTHER VARIABLES ##############
        
        ht = jet.pt.sum()
        
        min_dphiJetMet4 = np.arccos(np.cos(jet[:,:4].phi-met_phi)).min()
        
        leadingJets = jet[:,:2]
        dijet = leadingJets.choose(2)
        dphiDiJet = np.arccos(np.cos(dijet.i0.phi-dijet.i1.phi))
        
        min_dphiFatJetMet4 = np.arccos(np.cos(fatjet[:,:4].phi-met_phi)).min()

        ############## FILTERS ##############
        
        good_vertices = df["Flag_goodVertices"]
        tighthalo = df["Flag_globalSuperTightHalo2016Filter"]
        noise_filter = df["Flag_HBHENoiseFilter"]
        noise_isofilter = df["Flag_HBHENoiseIsoFilter"]
        ecal_deadcell = df["Flag_EcalDeadCellTriggerPrimitiveFilter"]
        bad_pfmuon = df["Flag_BadPFMuonFilter"]
        ee_badsc = df["Flag_eeBadScFilter"]
       
        ############## TRIGGERS ##############
        hlt_mu17 = df["HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL"]
        hlt_mu17_dz = df["HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ"]
        hlt_mu50 = df["HLT_Mu50"]
        hlt_mu55 = df["HLT_Mu55"]
        
        dimuon_tsel = (hlt_mu17 == 1) & (hlt_mu17_dz == 1) & (hlt_mu50 == 1) & (hlt_mu55 ==1)
        
#         hlt_pfmet_250 = df["HLT_PFMET250_HBHECleaned"]
#         hlt_pfmet_300 = df["HLT_PFMET300_HBHECleaned"]
#         hlt_pfmet1_200 = df["HLT_PFMETTypeOne200_HBHE_BeamHaloCleaned"]
#         hlt_pfmet_mht = df["HLT_PFMET120_PFMHT120_IDTight_PFHT60"]
#         hlt_pfmetNoMu_mhtNoMu = df["HLT_PFMETNoMu120_PFMHTNoMu120_IDTight_PFHT60"]
        
#         met_fsel = (good_vertices == 1) & (tighthalo == 1) & (noise_filter == 1) & (noise_isofilter == 1) & (ecal_deadcell == 1) & (bad_pfmuon == 1) & (ee_badsc == 1) 
#         met_tsel = (hlt_pfmet_250 == 1) | (hlt_pfmet_300 == 1) | (hlt_pfmet1_200 == 1) | (hlt_pfmet_mht == 1) | (hlt_pfmetNoMu_mhtNoMu == 1)
                
#         wtag_sel = ( wtag.counts>0 & (abs(wtag.msoftdrop-80)<30).any())
#         htag_sel = ( htag.counts>0 & (abs(htag.msoftdrop-125)<25).any())
#         stitchVar = 1 if dataset=='Data' else df["stitch"]
        
        ############## SELECTIONS ##############
        
        dilep_sel = ((electron.counts+muon.counts)==2)
        dilep_sf_sel = dilep_sel & ((electron.counts ==2 )|(muon.counts == 2))
        dilep_of_sel = dilep_sel & ((electron.counts ==1 )|(muon.counts == 1))
        
        ############## CUTFLOW ##############
        
        output['totalEvents']['all'] += len(df['weight'])
        processes = processesList
        weight = np.ones(len(df['weight'])) if dataset=='Data' else df['weight']
        lumi = 1 if dataset=='Data' else 137
        fullweight = weight * lumi
        
        cutflow = Cutflow(output, df, cfg, processes, weight=fullweight)
        cutflow.addRow( 'triggers',   (dimuon_tsel) )        
        cutflow.addRow('dilepton sf', (dilep_sf_sel))
#         cutflow.addRow( 'good_vertices',   (good_vertices==1) )
#         cutflow.addRow( 'tighthalo',   (tighthalo==1) )
#         cutflow.addRow( 'noise_filter',   (noise_filter==1) )
#         cutflow.addRow( 'noise_isofilter',   (noise_isofilter==1) )
#         cutflow.addRow( 'ecal_deadcell',   (ecal_deadcell==1) )
#         cutflow.addRow( 'bad_pfmuon',   (bad_pfmuon==1) )
#         cutflow.addRow( 'ee_badsc',   (ee_badsc==1) )

#         cutflow.addRow( 'triggers',   (met_tsel) )
        
#         cutflow.addRow( 'stitch',   (stitchVar ==1) )
        
#         cutflow.addRow( 'skim',   ((met_pt>200) & (skimjet.counts>1)) )
#         cutflow.addRow( 'Exactly 1 e or mu',   ((electron.counts+muon.counts)==1) )
#         cutflow.addRow( 'MET>250',     (met_pt>250) )
        
#         baseline = copy.deepcopy(cutflow.selection)
        
#         cutflow.addRow( 'N_fatjet>1',      (fatjet.counts>1) )
#         cutflow.addRow( 'min_dphiFatJetMet4', (min_dphiFatJetMet4>0.5))
#         cutflow.addRow( 'dphiDiFatJet', (dphiDiFatJet<2.5).all() ) # by using .all() I do not implicitely cut on the number of fat jets
#         cutflow.addRow( 'minmth>200',   (mth.min()>200) )
#         cutflow.addRow( 'njet veto',     (jet.counts<2))

#         vetoQCD = copy.deepcopy(cutflow.selection)
        
#         cutflow.addRow( 'N_wtag>0',     (wtag_sel), cumulative=False)
        
#         wtag_selection = copy.deepcopy(cutflow.selection)
        
#         cutflow.addRow( 'N_htag>0',     (htag_sel), cumulative=False)

#         htag_selection = copy.deepcopy(cutflow.selection)
        
#         cutflow.addRow( 'N_htag>0, N_wtag>0',     (htag_sel & wtag_sel))

#         signal_selection = cutflow.selection
        
        ############## HISTOGRAMS ##############
        
        output['Z_mass'].fill(dataset=dataset,mass=mm_mass.flatten(),weight=fullweight)
#         output['met_CR'].fill(dataset=dataset, pt=met_pt[vetoQCD].flatten(), weight=fullweight[vetoQCD])
#         output['met_W_CR'].fill(dataset=dataset, pt=met_pt[vetoQCD & wtag_sel].flatten(), weight=fullweight[vetoQCD & wtag_sel])
#         output['met_Higgs_CR'].fill(dataset=dataset, pt=met_pt[vetoQCD & htag_sel].flatten(), weight=fullweight[vetoQCD & htag_sel])
#         output['met_Higgs_W_CR'].fill(dataset=dataset, pt=met_pt[signal_selection].flatten(), weight=fullweight[signal_selection])

#         output['ht_CR'].fill(dataset=dataset, ht=ht[vetoQCD].flatten(), weight=fullweight[vetoQCD])
#         output['ht_W_CR'].fill(dataset=dataset, ht=ht[vetoQCD & wtag_sel].flatten(), weight=fullweight[vetoQCD & wtag_sel])
#         output['ht_Higgs_CR'].fill(dataset=dataset, ht=ht[vetoQCD & htag_sel].flatten(), weight=fullweight[vetoQCD & htag_sel])
#         output['ht_Higgs_W_CR'].fill(dataset=dataset, ht=ht[signal_selection].flatten(), weight=fullweight[signal_selection])
        
#         output['N_AK8_CR'].fill(dataset=dataset, multiplicity=fatjet[vetoQCD].counts, weight=fullweight[vetoQCD])
#         #output['W_pt_CR'].fill(dataset=dataset, pt=lead_wtag[vetoQCD].pt.flatten(), weight=fullweight[vetoQCD])
        #output['H_pt_CR'].fill(dataset=dataset, pt=lead_htag[vetoQCD].pt.flatten(), weight=fullweight[vetoQCD])
        #output['W_eta_CR'].fill(dataset=dataset, eta=lead_wtag[vetoQCD].eta.flatten(), weight=fullweight[vetoQCD])
        #output['H_eta_CR'].fill(dataset=dataset, eta=lead_htag[vetoQCD].eta.flatten(), weight=fullweight[vetoQCD])

#         output['N_AK8_W_CR'].fill(dataset=dataset, multiplicity=fatjet[vetoQCD & wtag_sel].counts, weight=fullweight[vetoQCD & wtag_sel])
#         output['W_pt_W_CR'].fill(dataset=dataset, pt=lead_wtag[vetoQCD & wtag_sel].pt.flatten(), weight=fullweight[vetoQCD & wtag_sel])
#         #output['H_pt_W_CR'].fill(dataset=dataset, pt=lead_htag[vetoQCD & wtag_sel].pt.flatten(), weight=fullweight[vetoQCD & wtag_sel])
        #output['W_eta_W_CR'].fill(dataset=dataset, eta=lead_wtag[vetoQCD & wtag_sel].eta.flatten(), weight=fullweight[vetoQCD & wtag_sel])
        #output['H_eta_W_CR'].fill(dataset=dataset, eta=lead_htag[vetoQCD & wtag_sel].eta.flatten(), weight=fullweight[vetoQCD & wtag_sel])

        #output['N_AK8_Higgs_CR'].fill(dataset=dataset, multiplicity=fatjet[vetoQCD & htag_sel].counts, weight=fullweight[vetoQCD & htag_sel])
        #output['W_pt_Higgs_CR'].fill(dataset=dataset, pt=lead_wtag[vetoQCD & htag_sel].pt.flatten(), weight=fullweight[vetoQCD & htag_sel])
        #output['H_pt_Higgs_CR'].fill(dataset=dataset, pt=lead_htag[vetoQCD & htag_sel].pt.flatten(), weight=fullweight[vetoQCD & htag_sel])
        #output['W_eta_Higgs_CR'].fill(dataset=dataset, eta=lead_wtag[vetoQCD & htag_sel].eta.flatten(), weight=fullweight[vetoQCD & htag_sel])
        #output['H_eta_Higgs_CR'].fill(dataset=dataset, eta=lead_htag[vetoQCD & htag_sel].eta.flatten(), weight=fullweight[vetoQCD & htag_sel])

#         output['N_AK8_Higgs_W_CR'].fill(dataset=dataset, multiplicity=fatjet[signal_selection].counts, weight=fullweight[signal_selection])
#         output['W_pt_Higgs_W_CR'].fill(dataset=dataset, pt=lead_wtag[signal_selection].pt.flatten(), weight=fullweight[signal_selection])
#         output['H_pt_Higgs_W_CR'].fill(dataset=dataset, pt=lead_htag[signal_selection].pt.flatten(), weight=fullweight[signal_selection])
#         output['W_eta_Higgs_W_CR'].fill(dataset=dataset, eta=lead_wtag[signal_selection].eta.flatten(), weight=fullweight[signal_selection])
#         output['H_eta_Higgs_W_CR'].fill(dataset=dataset, eta=lead_htag[signal_selection].eta.flatten(), weight=fullweight[signal_selection])

                
#         output['MET_pt_baseline'].fill(dataset=dataset, pt=met_pt[baseline].flatten(), weight=df['weight'][baseline]*cfg['lumi'])
#         output['HT_baseline'].fill(dataset=dataset, ht=ht[baseline].flatten(), weight=df['weight'][baseline]*cfg['lumi'])
#         output['mtb_min_baseline'].fill(dataset=dataset, mass=mtb[baseline].min().flatten(), weight=df['weight'][baseline]*cfg['lumi'])

#         output['MET_pt'].fill(dataset=dataset, pt=met_pt[vetoQCD].flatten(), weight=df['weight'][vetoQCD]*cfg['lumi'])
#         output['HT'].fill(dataset=dataset, ht=ht[vetoQCD].flatten(), weight=df['weight'][vetoQCD]*cfg['lumi'])
#         output['mtb_min'].fill(dataset=dataset, mass=mtb[vetoQCD].min().flatten(), weight=df['weight'][vetoQCD]*cfg['lumi'])
        
#         ## N jet and N b without selections on those
#         output['N_AK4'].fill(dataset=dataset, multiplicity=jet[baseline].counts, weight=df['weight'][baseline]*cfg['lumi'])
#         output['N_b'].fill(dataset=dataset, multiplicity=btag[baseline].counts, weight=df['weight'][baseline]*cfg['lumi'])       
#         output['N_W'].fill(dataset=dataset, multiplicity=htag[baseline].counts, weight=df['weight'][baseline]*cfg['lumi'])       
#         output['N_H'].fill(dataset=dataset, multiplicity=wtag[baseline].counts, weight=df['weight'][baseline]*cfg['lumi'])       
#         output['N_AK8'].fill(dataset=dataset, multiplicity=fatjet[baseline].counts, weight=df['weight'][baseline]*cfg['lumi'])       

#         #output['bb_deltaPhi'].fill(dataset=dataset, delta=bb_deltaPhi[baseline].flatten(), weight=df['weight'][baseline]*cfg['lumi'])
#         #output['bb_deltaR'].fill(dataset=dataset, delta=bb_deltaR[baseline].flatten(), weight=df['weight'][baseline]*cfg['lumi'])

#         output['min_dphiJetMet4'].fill(dataset=dataset, delta=min_dphiJetMet4[baseline].flatten(), weight=df['weight'][baseline]*cfg['lumi'])
#         output['dphiDiJet'].fill(dataset=dataset, delta=dphiDiJet[baseline].min().flatten(), weight=df['weight'][baseline]*cfg['lumi'])

#         ## Higgs and W pt
#         output['lead_AK8_pt'].fill(dataset=dataset, pt=fatjet[(baseline & (fatjet.counts>0))].pt.max().flatten(), weight=df['weight'][(baseline & (fatjet.counts>0))]*cfg['lumi'])
#         output['dphiDiFatJet'].fill(dataset=dataset, delta=dphiDiFatJet[(baseline & (fatjet.counts>1))].min().flatten(), weight=df['weight'][(baseline & (fatjet.counts>1))]*cfg['lumi'])

#         output['H_pt'].fill(dataset=dataset, pt=lead_htag[event_selection].pt.flatten(), weight=df['weight'][event_selection]*cfg['lumi'])
#         output['H_eta'].fill(dataset=dataset, eta=lead_htag[event_selection].eta.flatten(), weight=df['weight'][event_selection]*cfg['lumi'])

#         output['W_pt'].fill(dataset=dataset, pt=lead_wtag[event_selection].pt.flatten(), weight=df['weight'][event_selection]*cfg['lumi'])
#         output['W_eta'].fill(dataset=dataset, eta=lead_wtag[event_selection].eta.flatten(), weight=df['weight'][event_selection]*cfg['lumi'])

#         output['WH_deltaPhi'].fill(dataset=dataset, delta=wh_deltaPhi[event_selection].flatten(), weight=df['weight'][event_selection]*cfg['lumi'])
#         output['WH_deltaR'].fill(dataset=dataset, delta=wh_deltaR[event_selection].flatten(), weight=df['weight'][event_selection]*cfg['lumi'])

#         output['MET_pt_CR'].fill(dataset=dataset, pt=met_pt[event_selection].flatten(), weight=df['weight'][event_selection]*cfg['lumi'])
#         output['HT_CR'].fill(dataset=dataset, ht=ht[event_selection].flatten(), weight=df['weight'][event_selection]*cfg['lumi'])
#         output['mtb_min_CR'].fill(dataset=dataset, mass=mtb[event_selection].min().flatten(), weight=df['weight'][event_selection]*cfg['lumi'])

        return output

    def postprocess(self, accumulator):
        return accumulator

In [12]:
runLocal = True


if not runLocal:
    # Get the scheduler from the dask_cluster notebook
    from dask.distributed import Client, progress

    c = Client('tcp://169.228.130.5:27879')

    ## for dask
    exe_args = {
        'client': c,
        #'savemetrics': True,
    }
    exe = processor.dask_executor
    
else:
    ## for local
    exe_args = {
        'workers': 4,
        'function_args': {'flatten': False}
    }
    exe = processor.futures_executor

if not runLocal:
    print(c)

In [ ]:
overwrite = True
small = True

tag = 'v0.2.4'


fileset_all_2016   = {'Data': glob.glob('/hadoop/cms/store/user/mbryson/WH_hadronic/'+tag+'/DoubleMuon*2016*/*')
                + glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/'+tag+'/DoubleMuon*2016*/*'),
                'DYJets': glob.glob('/hadoop/cms/store/user/mbryson/WH_hadronic/'+tag+'/DYJetsToLL*Summer16*/*')
                + glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/'+tag+'/DYJetsToLL*Summer16*/*'),
                'TTJets': glob.glob('/hadoop/cms/store/user/mbryson/WH_hadronic/'+tag+'/TTJets_DiLept_Tune*Summer16*/*')
                + glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/'+tag+'/TTJets_DiLept_Tune*Summer16*/*'),
                'ttW': glob.glob('/hadoop/cms/store/user/mbryson/WH_hadronic/'+tag+'/TTWJets*Summer16*/*')
                + glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/'+tag+'/TTWJets*Summer16*/*'),
                'ttZ': glob.glob('/hadoop/cms/store/user/mbryson/WH_hadronic/'+tag+'/ttZJets*Summer16*/*')
                + glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/'+tag+'/ttZJets*Summer16*/*')
                 }

fileset_all_2016_sm   = {'Data': glob.glob('/hadoop/cms/store/user/mbryson/WH_hadronic/'+tag+'/DoubleMuon*2016*/*')[:2]
                + glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/'+tag+'/DoubleMuon*2016*/*')[:2],
                'DYJets': glob.glob('/hadoop/cms/store/user/mbryson/WH_hadronic/'+tag+'/DYJetsToLL*Summer16*/*')[:2]
                + glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/'+tag+'/DYJetsToLL*Summer16*/*')[:2],
                'TTJets': glob.glob('/hadoop/cms/store/user/mbryson/WH_hadronic/'+tag+'/TTJets_DiLept_Tune*Summer16*/*')[:2]
                + glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/'+tag+'/TTJets_DiLept_Tune*Summer16*/*')[:2],
                'ttW': glob.glob('/hadoop/cms/store/user/mbryson/WH_hadronic/'+tag+'/TTWJets*Summer16*/*')[:2]
                + glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/'+tag+'/TTWJets*Summer16*/*')[:2],
                'ttZ': glob.glob('/hadoop/cms/store/user/mbryson/WH_hadronic/'+tag+'/ttZJets*Summer16*/*')[:2]
                + glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/'+tag+'/ttZJets*Summer16*/*')[:2]
                 }


# load the config and the cache
cfg = loadConfig()

cacheName = 'WH_small' if small else 'WH'

# histograms
histograms = []
histograms += ['Z_mass']

# initialize cache
cache = dir_archive(os.path.join(os.path.expandvars(cfg['caches']['base']), cfg['caches'][cacheName]), serialized=True)
if not overwrite:
    cache.load()

if cfg == cache.get('cfg') and histograms == cache.get('histograms') and cache.get('simple_output'):
    output = cache.get('simple_output')

else:
    # Run the processor

    if small:
        fileset = fileset_all_2016_sm
        workers = 4
    else:
        fileset = fileset_all_2016
        workers = 16
    
        
    output = processor.run_uproot_job(fileset,
                                      treename='Events',
                                      processor_instance=analysisProcessor(),
                                      executor=exe,
                                      executor_args=exe_args,
                                      chunksize=250000,
                                      #chunksize=100000,
                                     )
    cache['fileset']        = fileset
    cache['cfg']            = cfg
    cache['histograms']     = histograms
    cache['simple_output']  = output
    cache.dump()

HBox(children=(HTML(value='Processing'), FloatProgress(value=0.0, max=28.0), HTML(value='')))

/home/users/mbryson/CMSSW_10_2_9/src/tW_scattering/coffeaEnv/lib/python3.6/site-packages/awkward/array/jagged.py:1043: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/users/mbryson/CMSSW_10_2_9/src/tW_scattering/coffeaEnv/lib/python3.6/site-packages/awkward/array/jagged.py:1043: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)


/home/users/mbryson/CMSSW_10_2_9/src/tW_scattering/coffeaEnv/lib/python3.6/site-packages/awkward/array/jagged.py:1043: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/users/mbryson/CMSSW_10_2_9/src/tW_scattering/coffeaEnv/lib/python3.6/site-packages/awkward/array/jagged.py:1043: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
# Cutflow
from Tools.helpers import getCutFlowTable

processes = processesList
lines     = ['entry']
lines    += linesList
df        = getCutFlowTable(output, processes=processes, lines=lines, significantFigures=4)
df

In [ ]:
# Efficiencies
df = getCutFlowTable(output, processes=processes, lines=lines, significantFigures=3, absolute=False)
df

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [ ]:
from plots.helpers import *
bins = {
    'Z_mass_coarse':   {'axis':'mass',   'overflow': 'over', 'bins': hist.Bin("mass",  r"Z Boson Mass (GeV)", 1000, 0, 2000)},
#     'N_AK4':    {'axis': 'multiplicity',  'overflow':'over',  'bins': hist.Bin('multiplicity', r'$N_{AK4 jet}$', 6, -0.5, 5.5)},
#     'N_AK4_SR':    {'axis': 'multiplicity',  'overflow':'over',  'bins': hist.Bin('multiplicity', r'$N_{AK4 jet}$', 6, -0.5, 5.5)},
#     'N_AK8':    {'axis': 'multiplicity',  'overflow':'over',  'bins': hist.Bin('multiplicity', r'$N_{AK8 jet}$', 5, -0.5, 4.5)},
#     'N_b':      {'axis': 'multiplicity',  'overflow':'over',  'bins': hist.Bin('multiplicity', r'$N_{b-tag}$', 5, -0.5, 4.5)},
#     'N_H':      {'axis': 'multiplicity',  'overflow':'over',  'bins': hist.Bin('multiplicity', r'$N_{H-tag}$', 5, -0.5, 4.5)},
#     'N_W':      {'axis': 'multiplicity',  'overflow':'over',  'bins': hist.Bin('multiplicity', r'$N_{W-tag}$', 5, -0.5, 4.5)},

#     'MET_pt':   {'axis': 'pt',      'overflow':'over',  'bins': hist.Bin('pt', r'$p_{T}^{miss}\ (GeV)$', 20, 0, 800)},
#     'MET_ptCoarse':   {'axis': 'pt',      'overflow':'over',  'bins': hist.Bin('pt', r'$p_{T}^{miss}\ (GeV)$', 5, 200, 700)},
#     'HT':       {'axis': 'ht',      'overflow':'over',  'bins': hist.Bin('pt', r'$H_{T} (AK4 jets) \ (GeV)$', 25, 0, 2000)},    
#     'HT_Coarse':       {'axis': 'ht',      'overflow':'over',  'bins': hist.Bin('pt', r'$H_{T} (AK4 jets) \ (GeV)$', 5, 0, 500)},    
#     'W_pt':     {'axis': 'pt',      'overflow':'over',  'bins': hist.Bin('pt', r'$p_{T} (W-tag)$', 8, 200, 600)},
#     'W_eta':    {'axis': 'eta',     'overflow':'over',  'bins': hist.Bin('eta', r'$\eta (W-tag)$', 15, -5.5, 5.5)},
#     'H_pt':     {'axis': 'pt',      'overflow':'over',  'bins': hist.Bin('pt', r'$p_{T} (H-tag)$', 8, 200, 600)},
#     'H_eta':    {'axis': 'eta',     'overflow':'over',  'bins': hist.Bin('eta', r'$\eta (H-tag)$', 15, -5.5, 5.5)},

#     'dphiDiFatJet': {'axis': 'delta',          'overflow':'over',  'bins': hist.Bin('delta', r'$\Delta \phi (AK8)$', 30, 0, 3)},
#     'dphiDiJet':    {'axis': 'delta',          'overflow':'over',  'bins': hist.Bin('delta', r'$\Delta \phi (AK4)$', 30, 0, 3)},
#     'WH_deltaPhi':  {'axis': 'delta',          'overflow':'over',  'bins': hist.Bin('delta', r'$\Delta \phi (WH)$', 6, 0, 3)},
#     'WH_deltaR':    {'axis': 'delta',          'overflow':'over',  'bins': hist.Bin('delta', r'$\Delta R (WH)$', 10, 0, 5)},
#     'bb_deltaPhi':  {'axis': 'delta',          'overflow':'over',  'bins': hist.Bin('delta', r'$\Delta \phi (bb)$', 30, 0, 3)},
#     'bb_deltaR':    {'axis': 'delta',          'overflow':'over',  'bins': hist.Bin('delta', r'$\Delta R (bb)$', 10, 0, 5)},
#     'min_dphiJetMet4': {'axis': 'delta',          'overflow':'over',  'bins': hist.Bin('delta', r'$\Delta \phi (j, p_{T}^{miss})$', 30, 0, 3)},
        
#     'mtb_min':      {'axis': 'mass',  'overflow':'over',  'bins': hist.Bin('pt', r'$min M_{T} (b, p_{T}^{miss}) \ (GeV)$', 25, 0, 500)},
#     'lead_AK8_pt':  {'axis': 'pt',    'overflow':'over',  'bins': hist.Bin('pt', r'$p{T} (lead. AK8) \ (GeV)$', 20, 0, 1000)},
    }

In [ ]:
from plots.helpers import *

def saveFig( fig, ax, rax, path, name, scale='linear', shape=False, y_max=-1 ):
    outdir = os.path.join(path,scale)
    finalizePlotDir(outdir)
    ax.set_yscale(scale)
    ax.set_ylabel('Events')

    if scale == 'linear':
        if y_max<0: #or True:
            pass
        else:
            ax.set_ylim(0, 1 if shape else 1.2*y_max)
    else:
        if y_max<0 and not shape:
            pass
        else:
            ax.set_ylim(0.000005 if shape else 0.05, 3 if shape else 300*y_max)

    handles, labels = ax.get_legend_handles_labels()
    new_labels = []
    for handle, label in zip(handles, labels):
        #print (handle, label)
        try:
            new_labels.append(my_labels[label])
            if not label=='pseudodata':
                handle.set_color(colors[label])
        except:
            pass

    if rax:
        plt.subplots_adjust(hspace=0)
        rax.set_ylabel('Obs./Pred.')
        rax.set_ylim(0.5,1.5)

    ax.legend(title='',ncol=2,handles=handles, labels=new_labels, frameon=False)

    fig.text(0., 0.995, '$\\bf{CMS}$', fontsize=20,  horizontalalignment='left', verticalalignment='bottom', transform=ax.transAxes )
    fig.text(0.15, 1., '$\\it{Simulation}$', fontsize=14, horizontalalignment='left', verticalalignment='bottom', transform=ax.transAxes )
    fig.text(0.8, 1., '13 TeV', fontsize=14, horizontalalignment='left', verticalalignment='bottom', transform=ax.transAxes )

    fig.savefig(os.path.join(outdir, "{}.pdf".format(name)))
    fig.savefig(os.path.join(outdir, "{}.png".format(name)))
    #ax.clear()

In [ ]:
#Histos I want to save
histos = [["Z_mass", "Z_mass_coarse"]]
# histos = [["met_CR", "MET_ptCoarse"],
#           ["met_W_CR", "MET_ptCoarse"],
#           ["met_Higgs_CR", "MET_ptCoarse"],
#           ["met_Higgs_W_CR", "MET_ptCoarse"],
          
#           ["ht_CR", "HT_Coarse"],
#           ["ht_W_CR", "HT_Coarse"],
#           ["ht_Higgs_CR", "HT_Coarse"],
#           ["ht_Higgs_W_CR", "HT_Coarse"],
          
#           ["N_AK8_CR", "N_AK8"],
#           ["N_AK8_W_CR", "N_AK8"],
#           ["N_AK8_Higgs_CR", "N_AK8"],
#           ["N_AK8_Higgs_W_CR", "N_AK8"],
          
#           ["W_pt_W_CR", "W_pt"],
#           #["H_pt_W_CR", "H_pt"],
#           ["W_eta_W_CR", "W_eta"],
#           #["H_eta_W_CR", "H_eta"],
          
#           #["W_pt_Higgs_CR", "W_pt"],
#           ["H_pt_Higgs_CR", "H_pt"],
#           #["W_eta_Higgs_CR", "W_eta"],
#           ["H_eta_Higgs_CR", "H_eta"],
          
#           ["W_pt_Higgs_W_CR", "W_pt"],
#           ["H_pt_Higgs_W_CR", "H_pt"],
#           ["W_eta_Higgs_W_CR", "W_eta"],
#           ["H_eta_Higgs_W_CR", "H_eta"],
#         ]

In [ ]:
#Plotting aesthetics

lineopts = {
    'color': 'r',
    'linewidth': '3'}

data_err_opts = {
    'linestyle': 'none',
    'marker': '_',
    'markersize': 10.,
    'color': 'r',
    'elinewidth': 1}

data_err_opts_rat = {
    'linestyle': 'none',
    'marker': '.',
    'markersize': 10.,
    'color': 'k',
    'elinewidth': 1}

fillopts2 = {
    'edgecolor': (0,0,0,0.3),
    'facecolor': [('#989C94'),('#6A0136'),('#FF5714'),('#FFCA3A')]  
}

In [ ]:
histogram = output['Z_mass']
ax = hist.plot1d(histogram,overlay="dataset", stack=False)

In [ ]:
# Make some of the plots

plotDir = '/home/users/mbryson/public_html/dump/WH/dilep/'
finalizePlotDir(plotDir)


for plot in histos:

    name = plot[0]
    binName = plot[1]
    histogram = output[name]

    axis = bins[binName]['axis']
    histogram = histogram.rebin(axis, bins[binName]['bins'])

    y_max = histogram.sum("dataset").values(overflow='all')[()].max()
    y_over = histogram.sum("dataset").values(overflow='all')[()][-1]

    import re

    bkg = re.compile('(?!Data)')
    
    background = histogram[bkg]
    data = histogram['Data']

    #fig, ax = plt.subplots(1,1,figsize=(7,7))
    fig, (ax, rax) = plt.subplots(nrows=2,ncols=1, figsize=(7,7),
        gridspec_kw={"height_ratios": (3, 1)}, sharex=True)
    
    # get axes
    hist.plot1d(background, overlay="dataset", ax=ax, stack=True, 
                overflow=bins[binName]['overflow'], clear=False, fill_opts=fillopts2, 
                error_opts=error_opts, order=['DYJets','TTJets', 'ttW', 'ttZ']) #error_opts??
    hist.plot1d(data, overlay="dataset", ax=ax, stack=False, 
                overflow=bins[binName]['overflow'], error_opts=data_err_opts_rat, 
                clear=False)

    hist.plotratio(num=data.sum('dataset'), denom=background.sum('dataset'), ax=rax,
                   error_opts = data_err_opts_rat, denom_fill_opts={}, guide_opts={}, 
                   unc='num', overflow = 'over')

    for l in ['log', 'linear']:
        saveFig(fig, ax, rax, plotDir, name, scale=l, shape=False, y_max=y_max)


In [ ]:
output['Z_mass'].sum('dataset').values()
